In [ ]:
# Ячейка для мгновенного обновления модулей
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
from dataclasses import dataclass

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

import warnings
warnings.simplefilter('ignore')

# 1. Алгоритм Нидлмана-Вунша

In [147]:
@dataclass
class NeedlmanWunchAln:
    seq1: str
    seq2: str
    match: float = 1.0
    mismatch: float = 0.0
    open_gap: float = -2.0
    extend_gap: float = -0.5

    def __post_init__(self):
        self.alphabet = sorted(set(self.seq2) | set(self.seq1))
        self.vocab_length = len(self.alphabet)

        self.letter_to_idx = dict(enumerate(self.alphabet))
        self.idx_to_letter = {v: k for k, v in self.letter_to_idx.items()}

        self.matrix = np.zeros((len(self.seq1) + 1, len(self.seq2) + 1))

        # [0: from diag, 1: from left, 2: from up]
        self.matrix_pass = np.zeros((len(self.seq1) + 1, len(self.seq2) + 1))
    
    def forward_pass(self):
        self.matrix[0, :] = self.open_gap * np.arange(len(self.seq2) + 1)
        self.matrix[:, 0] = self.open_gap * np.arange(len(self.seq1) + 1)

        for i in range(1, len(self.seq1) + 1):
            for j in range(1, len(self.seq2) + 1):
                match_score = self.match if self.seq1[i - 1] == self.seq2[j - 1] else self.mismatch
                diag = self.matrix[i - 1, j - 1] + match_score

                # Обрабатываем гэпы
                if self.matrix[i, j - 1] == self.matrix[i, j - 2] + self.extend_gap:
                    left = self.matrix[i, j - 1] + self.extend_gap
                else:
                    left = self.matrix[i, j - 1] + self.open_gap

                if self.matrix[i - 1, j] == self.matrix[i - 2, j] + self.extend_gap:
                    up = self.matrix[i - 1, j] + self.extend_gap
                else:
                    up = self.matrix[i - 1, j] + self.open_gap

                self.matrix[i, j] = max(diag, left, up)
                self.matrix_pass[i, j] = np.argmax([diag, left, up])
        
    def backward_pass(self):
        seq1_aln = ''
        seq2_aln = ''

        i = len(self.seq1)
        j = len(self.seq2)

        self.score = self.matrix[-1, -1].item()
        
        while i > 0 or j > 0:  # continue until one of the sequences is fully processed

            # Diagonal (match/mismatch)
            if i > 0 and j > 0 and self.matrix_pass[i, j] == 0:
                seq1_aln += self.seq1[i - 1]
                seq2_aln += self.seq2[j - 1]

                i -= 1
                j -= 1
            
            # Left (gap in seq1)
            elif j > 0 and (i == 0 or self.matrix_pass[i, j] == 1):
                seq1_aln += '-'
                seq2_aln += self.seq2[j - 1]
                j -= 1
            
            # Up (gap in seq2)
            elif i > 0 and (j == 0 or self.matrix_pass[i, j] == 2):
                seq1_aln += self.seq1[i - 1]
                seq2_aln += '-'

                i -= 1
        
        return {'seq1': seq1_aln[::-1], 
                'seq2': seq2_aln[::-1], 
                'score': self.score}
    
    def align(self):
        self.forward_pass()
        res = self.backward_pass()
        return res

In [148]:
seq1 = "GTACGATCG"
seq2 = "GTCGACG"

In [149]:
len(seq1)

9

In [152]:
%%time
pair_aligner = NeedlmanWunchAln(seq1=seq1, seq2=seq2, match=2, mismatch=-1, open_gap=-2, extend_gap=-0.5)
pair_aligner.align()

CPU times: user 3.12 ms, sys: 0 ns, total: 3.12 ms
Wall time: 2.55 ms


{'seq1': 'GTACGATCG', 'seq2': 'GT-CGA-CG', 'score': 10.0}

In [153]:
%%time
alignments = pairwise2.align.globalms(seq1, seq2, 
                                     match=2, mismatch=-1, 
                                     open=-2, extend=-0.5)

alignments[0]

CPU times: user 527 μs, sys: 0 ns, total: 527 μs
Wall time: 576 μs


Alignment(seqA='GTACGATCG', seqB='GT-CGA-CG', score=10.0, start=0, end=9)